The 'ml' service should be up and running and consuming messages that are published to the kafka topic from the 'generator' service set in the docker-compose.yaml file.


In [ ]:
# jupyterlab_vim

In [ ]:
## output of ml inference
from kafka import KafkaConsumer
from kafka.admin import KafkaAdminClient

bootstrap_servers = "kafka-broker:9092"

admin_client = KafkaAdminClient(
    bootstrap_servers=bootstrap_servers,
    client_id="demo",
)
topic = "message_emotions"

In [ ]:
producer_topic = os.getenv("KAFKA_PRODUCER_TOPIC", topic)
producer_topic = KafkaConsumer(
    producer_topic,
    bootstrap_servers=["kafka-broker:9092"],
    auto_offset_reset="earliest",
    enable_auto_commit=False,
)

result_batch = producer_topic.poll(timeout_ms=1000)
for partition, messages in result_batch.items():
    for message in messages:
        print(message.value.decode("utf-8"))
producer_topic.close()

In [ ]:
# ingest this new topic into rising wave for further analysis

In [ ]:
! export PYTHONPATH=../risingwave:$PYTHONPATH && python ../risingwave/src/main.py --topic='message_emotions' --bootstrap-servers='kafka-broker:9092' --schema='{"id": "VARCHAR", "created_at": "TIMESTAMP", "message": "VARCHAR", "emotion":"VARCHAR"}' --source=kafka

In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(host="risingwave", port=4566, user="root", dbname="dev")
conn.autocommit = True

with conn.cursor() as cur:
    cur.execute(
        f"SELECT kafka_timestamp, id, created_at, message, emotion FROM {topic} LIMIT 10;"
    )
    values = cur.fetchall()

In [ ]:
df = pd.DataFrame(
    values, columns=["timestamp", "id", "created_at", "message", "emotion"]
)
print(df.tail())

In [ ]:
with conn.cursor() as cur:
    cur.execute(
        f"SELECT emotion, count(*) FROM {topic} group by emotion order by emotion;"
    )
    values = cur.fetchall()
summary = pd.DataFrame(values, columns=["emotion", "count"])

In [ ]:
print(summary["count"].sum())
summary

In [ ]:
with conn.cursor() as cur:
    cur.execute(f"select * from {topic} LIMIT 10;")
    values = cur.fetchall()
view = pd.DataFrame(
    values, columns=["emotions", "id", "created_at", "message", "timestamp", "metadata"]
)
print(view.head())
metadata = view["metadata"].loc[0]
print(type(metadata))
print(metadata)